# Лабораторная работа №3
### Простые системы классификации изображений на основе сверточных нейронных сетей
Ход работы:

Подготовка датасета и набора лейблов


In [ ]:
!rm -R ./samples/*

In [ ]:
#Подгружаем данные
# Download ImageNet labels
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

--2023-03-14 13:26:37--  https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10472 (10K) [text/plain]
Saving to: ‘imagenet_classes.txt’

imagenet_classes.tx 100%[===================>]  10.23K  --.-KB/s    in 0s      

2023-03-14 13:26:37 (34.2 MB/s) - ‘imagenet_classes.txt’ saved [10472/10472]



AlexNet:

In [ ]:
#
#Основной скрипт
#
from torchvision import transforms
from torchvision import models
import torch
from PIL import Image

from torch.profiler import profile, record_function, ProfilerActivity

import time
from os import walk


#
# Функция для получения самых повторяемых меток
#
def sortList(labels_list):
  li = sorted(set(labels_list), key = lambda ele: labels_list.count(ele))
  return list(reversed(li))[:5]


#
# Основной процесс
#

def main_process(input_batch):

  
  alexNet = models.alexnet(pretrained=True)

  alexNet.eval()

  # Читаем файл с категориями. По идее, 1000 классов
  with open("imagenet_classes.txt", "r") as f:
      categories = [s.strip() for s in f.readlines()]

  #Время исполнения начало
  start = time.time()

  with profile(activities=[ProfilerActivity.CPU],
        profile_memory=True, record_shapes=True) as prof:

    #Считываем изображения
    current_item  = 1
    labels_list = []
    for current_data in input_batch:

      print(">>>>> Item {} :".format(current_item))

      with torch.no_grad():
          output = alexNet(current_data)

      # Нормализуем скоры через софтмакс для получения вероятностей. 
      probabilities = torch.nn.functional.softmax(output[0], dim=0)
      #print(probabilities)


      # Для каждого изображения показываем топ-5
      top5_prob, top5_catid = torch.topk(probabilities, 5)
      for i in range(top5_prob.size(0)):
          labels_list.append(categories[top5_catid[i]])
          print(categories[top5_catid[i]], top5_prob[i].item())

      current_item = current_item +1


  #Время исполнения конец
  end = time.time()
  #Печатаем время исполнения
  print("Time consuption: {} s".format((end-start)))
  print(sortList(labels_list))
  print("-"*10)
  print(prof.key_averages().table(sort_by="self_cpu_memory_usage", row_limit=10))


#
# НАЧАЛО СКРИПТА
#

input_batch = []

#Пока без проверки :)
filenames = next(walk("./samples"), (None, None, []))[2]

#Загружаем изображения, нормируем их, загружаем в микробатчик
for image_name in filenames:
  input_image = Image.open("./samples/"+image_name)
  preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
  ])
  input_tensor = preprocess(input_image)
  current_batch = input_tensor.unsqueeze(0)
  input_batch.append(current_batch)

main_process(input_batch)

>>>>> Item 1 :
sorrel 0.9989705085754395
Rhodesian ridgeback 0.0006189715350046754
ox 0.00010281256982125342
worm fence 0.00010224067227682099
Doberman 5.315503949532285e-05
>>>>> Item 2 :
kuvasz 0.5143368244171143
ram 0.18424054980278015
bighorn 0.1356443613767624
Great Pyrenees 0.0375874862074852
Saluki 0.030267126858234406
>>>>> Item 3 :
American egret 0.2801472842693329
Ibizan hound 0.2799360454082489
crane 0.08426295965909958
sulphur-crested cockatoo 0.07497162371873856
little blue heron 0.048415012657642365
>>>>> Item 4 :
sorrel 0.9999074935913086
hartebeest 3.786500383284874e-05
redbone 3.546905281837098e-05
ox 9.040592885867227e-06
Rhodesian ridgeback 6.962882252992131e-06
>>>>> Item 5 :
sorrel 0.7403348088264465
Great Dane 0.07826816290616989
Arabian camel 0.04402286559343338
Rhodesian ridgeback 0.028390787541866302
Irish setter 0.025267813354730606
>>>>> Item 6 :
whippet 0.40346869826316833
American Staffordshire terrier 0.23253484070301056
Saluki 0.0634893998503685
bull mast

GoogLeNet:

In [ ]:
#
#Основной скрипт
#
from torchvision import transforms
from torchvision import models
import torch
from PIL import Image

from torch.profiler import profile, record_function, ProfilerActivity

import time
from os import walk


#
# Функция для получения самых повторяемых меток
#
def sortList(labels_list):
  li = sorted(set(labels_list), key = lambda ele: labels_list.count(ele))
  return list(reversed(li))[:5]


#
# Основной процесс
#

def main_process(input_batch):

  
  googlenet = models.googlenet(pretrained=True)

  googlenet.eval()

  # Читаем файл с категориями. По идее, 1000 классов
  with open("imagenet_classes.txt", "r") as f:
      categories = [s.strip() for s in f.readlines()]

  #Время исполнения начало
  start = time.time()

  with profile(activities=[ProfilerActivity.CPU],
        profile_memory=True, record_shapes=True) as prof:

    #Считываем изображения
    current_item  = 1
    labels_list = []
    for current_data in input_batch:

      print(">>>>> Item {} :".format(current_item))

      with torch.no_grad():
          output = googlenet(current_data)

      # Нормализуем скоры через софтмакс для получения вероятностей. 
      probabilities = torch.nn.functional.softmax(output[0], dim=0)
      #print(probabilities)


      # Для каждого изображения показываем топ-5
      top5_prob, top5_catid = torch.topk(probabilities, 5)
      for i in range(top5_prob.size(0)):
          labels_list.append(categories[top5_catid[i]])
          print(categories[top5_catid[i]], top5_prob[i].item())

      current_item = current_item +1


  #Время исполнения конец
  end = time.time()
  #Печатаем время исполнения
  print("Time consuption: {} s".format((end-start)))
  print(sortList(labels_list))
  print("-"*10)
  print(prof.key_averages().table(sort_by="self_cpu_memory_usage", row_limit=10))


#
# НАЧАЛО СКРИПТА
#

input_batch = []

#Пока без проверки :)
filenames = next(walk("./samples"), (None, None, []))[2]

#Загружаем изображения, нормируем их, загружаем в микробатчик
for image_name in filenames:
  input_image = Image.open("./samples/"+image_name)
  preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
  ])
  input_tensor = preprocess(input_image)
  current_batch = input_tensor.unsqueeze(0)
  input_batch.append(current_batch)

main_process(input_batch)


>>>>> Item 1 :
sorrel 0.9943335056304932
Ibizan hound 0.0013858381425961852
worm fence 0.0008118535624817014
ox 0.000508662371430546
basenji 0.0004782849573530257
>>>>> Item 2 :
Saluki 0.6957271099090576
borzoi 0.08791200816631317
Afghan hound 0.017942283302545547
white wolf 0.013728471472859383
whippet 0.012917846441268921
>>>>> Item 3 :
whippet 0.2501278817653656
Italian greyhound 0.07310396432876587
borzoi 0.04777490720152855
Ibizan hound 0.04129253700375557
Saluki 0.024997156113386154
>>>>> Item 4 :
sorrel 0.9270010590553284
hartebeest 0.009087903425097466
Rhodesian ridgeback 0.003727186005562544
velvet 0.0025576078332960606
ox 0.0017989279003813863
>>>>> Item 5 :
sorrel 0.7142813801765442
hartebeest 0.055016808211803436
Rhodesian ridgeback 0.008585271425545216
Arabian camel 0.006921200081706047
plow 0.006815693806856871
>>>>> Item 6 :
whippet 0.26130983233451843
Great Dane 0.14784479141235352
sorrel 0.1464552879333496
horse cart 0.056728098541498184
Italian greyhound 0.01664858125

Теперь ResNet50, тоже тренировали на ImageNet
https://pytorch.org/hub/pytorch_vision_resnet/ 

In [ ]:
#
#Основной скрипт
#
from torchvision import transforms
from torchvision import models
import torch
from PIL import Image

from torch.profiler import profile, record_function, ProfilerActivity

import os

import time
from os import walk


#
# Функция для получения самых повторяемых меток
#
def sortList(labels_list):
  li = sorted(set(labels_list), key = lambda ele: labels_list.count(ele))
  return list(reversed(li))[:5]


#
# Основной процесс
#

def main_process(input_batch):

  
  resnet = models.resnet50(pretrained=True)

  resnet.eval()

  # Читаем файл с категориями. По идее, 1000 классов
  with open("imagenet_classes.txt", "r") as f:
      categories = [s.strip() for s in f.readlines()]

  #Время исполнения начало
  start = time.time()

  with profile(activities=[ProfilerActivity.CPU],
        profile_memory=True, record_shapes=True) as prof:

    #Считываем изображения
    current_item  = 1
    labels_list = []
    for current_data in input_batch:

      #print(">>>>> Item {} :".format(current_item))

      # Если можем, то заливаем на GPU
      if torch.cuda.is_available():
          current_data = current_data.to('cuda')
          resnet.to('cuda')

      with torch.no_grad():
          output = resnet(current_data)

      # Нормализуем скоры через софтмакс для получения вероятностей. 
      probabilities = torch.nn.functional.softmax(output[0], dim=0)
      #print(probabilities)


      # Для каждого изображения показываем топ-5
      top5_prob, top5_catid = torch.topk(probabilities, 5)
      for i in range(top5_prob.size(0)):
          labels_list.append(categories[top5_catid[i]])
          #print(categories[top5_catid[i]], top5_prob[i].item())

      current_item = current_item +1


  #Время исполнения конец
  end = time.time()
  #Печатаем время исполнения
  print("Time consuption: {} s".format((end-start)))
  print(sortList(labels_list))
  print("-"*10)
  print(prof.key_averages().table(sort_by="self_cpu_memory_usage", row_limit=10))


#
# НАЧАЛО СКРИПТА
#

input_batch = []

#Пока без проверки :)
filenames = next(walk("./samples"), (None, None, []))[2]

#Загружаем изображения, нормируем их, загружаем в микробатчик
for image_name in filenames:
  input_image = Image.open("./samples/"+image_name)
  preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
  ])
  input_tensor = preprocess(input_image)  
  current_batch = input_tensor.unsqueeze(0)
  input_batch.append(current_batch)

main_process(input_batch)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

Time consuption: 17.256290197372437 s
['sorrel', 'Saluki', 'hartebeest', 'worm fence', 'whippet']
----------
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                      aten::empty         0.64%      60.004ms         0.64%      60.004ms       3.069us       2.62 Gb       2.62 Gb         19550  
                    aten::resize_         0.15%      13.879ms         0.15%      13.879ms       8.412us       1.08 Gb       1.08 Gb          1650  
       aten::_slow_conv2d_forward        32.64%        3.045s        32.95%        3.073s       1.862ms       1.53 Gb     464.16 Mb          1650  
   

Список доступных моделей- https://pytorch.org/vision/stable/models.html
ВАЖНО ЧИТАТЬ ПРО ЭТИ МОДЕЛИ, ЧТОБЫ ОНИ ОБУЧАЛИСЬ НА ОДНОМ И ТОМ ЖЕ ДАТАСЕТЕ, например - ImageNet(соревнование для комп зрения)

GoogleLeNet(статья https://habr.com/ru/post/301084/) 

Она выиграла ImageNet recognition challenge в 2014-м году с результатом 6.67% top 5 error. Напомню, top 5 error — метрика, в которой алгоритм может выдать 5 вариантов класса картинки и ошибка засчитывается, если среди всех этих вариантов нет правильного. Всего в тестовом датасете 150K картинок и 1000 категорий

ТУТ ЕСТЬ ПОЯСНЕНИЕ
https://medium.com/@jubergandharv/googles-lenet-inception-net-f256c2976955 

Улучшение архитектур  ZFNet (The winner in 2013) and AlexNet (The winner in 2012)

Фишка - во внедрении т.н.Inception-модели, т.е. слабо-связанных блоков

КАРТИНКА С АРХИТЕКТУРОЙ http://joelouismarino.github.io/images/blog_images/blog_googlenet_keras/googlenet_diagram.png 